# Smart Products Classification example usecase

This notebook shows an example use case for classification using the Transfer Learning Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pretrained model](#head-1)
    1. [Split the dataset into train/test/val and do augmentation](#head-1-1)
    2. [Download pre-trained model](#head-1-2)
2. [Provide training specfication](#head-2)
3. [Run TLT training](#head-3)
4. [Evaluate trained models](#head-4)
5. [Visualize inferences](#head-5)
6. [Export and Deploy!](#head-6)
    1. [Int8 Optimization](#head-6-1)
    2. [Generate TensorRT engine](#head-6-2)

## 0. Setup env variables <a class="anchor" id="head-0"></a>

In [ ]:
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/DO_NOT_DELETE
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/DO_NOT_DELETE/data
%env SPECS_DIR=/workspace/tlt-experiments/DO_NOT_DELETE/specs
%env ZIP_DIR=/workspace/tlt-experiments
%env KEY=YOUR_KEY

## 1. Prepare datasets and pre-trained model <a class="anchor" id="head-1"></a>

Delete existing Folders

In [ ]:
!rm -rf /workspace/tlt-experiments/DO_NOT_DELETE/data
!rm -rf /workspace/tlt-experiments/DO_NOT_DELETE/classification

We will be using the SmartProducts Dataset! :)  
The Dataset has to be at: **/workspace/tlt-experiments/data/SmartProducts.zip** (from inside Docker)

Define the classes you want to use for Classification.  
**NOTE: Only take classes which consists one and only one object per image!**  
With the classes below, you should reach accuracy > **95%!**

In [ ]:
CLASSES = ['kellogs_variety', 'gruenespargeln', 'kraeutershampoo', 'kamillentee']

### A. Split the dataset into train/test/val and do augmentation <a class="anchor" id="head-1-1"></a>

In [ ]:
import zipfile
import os
import cv2
import numpy as np
import keras
import random
from tqdm import tqdm_notebook, tqdm
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator

class ImageHandler():
    def __init__(self, do_augmentation):
        self.image_paths = {}
        self.split_image_paths = {}
        self.data_archive = None
        self.do_augmentation = do_augmentation
        self.datagen = ImageDataGenerator(horizontal_flip=True,
                                          vertical_flip=True,
                                          width_shift_range=0.3,
                                          height_shift_range=0.3)
        
    def process(self, save_path):
        for class_name in tqdm_notebook(list(self.image_paths.keys()), desc='Classes'):
            for split in tqdm_notebook(['train', 'val', 'test'], desc='Splits', leave=False):
                for image_file in tqdm_notebook(self.split_image_paths[class_name][split], desc='Files', leave=False):
                    img = self.__load_image(image_path=image_file)
                    img = self.__allign_horizontal(img=img)
                    img = self.__resize(img=img)
                    if split == 'train' and self.do_augmentation:
                        sample = np.expand_dims(img, 0)
                        iterater = self.datagen.flow(sample, batch_size=1)
                        for idx in range(6):
                            img = iterater.next().squeeze()
                            img = img.astype('uint8')
                            self.__save_image('{}/{}/{}'.format(save_path, split, image_file), img, idx)
                    else:
                        self.__save_image('{}/{}/{}'.format(save_path, split, image_file), img)
    
    def shuffle(self):
        for class_name in CLASSES:
            random.shuffle(self.image_paths[class_name])
    
    def add_data_archive(self, path):
        self.data_archive = zipfile.ZipFile(path, 'r')
        
    def add_image(self, class_name, image):
        if class_name not in self.image_paths:
            self.image_paths[class_name] = []
        self.image_paths[class_name].append(image)
        
    def __load_image(self, image_path):
        return self.__bytes_to_numpy(img_as_bytes=self.data_archive.read(image_path))
    
    def __bytes_to_numpy(self, img_as_bytes):
        return cv2.imdecode(np.frombuffer(img_as_bytes, np.uint8), 1)
    
    def __allign_horizontal(self, img):
        if img.shape[0] > img.shape[1]:
            return cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
        else:
            return img
            
    def __resize(self, img):
        return cv2.resize(img, (320, 240), interpolation = cv2.INTER_AREA)
        
    def __save_image(self, path, img, idx=0):
        Path(os.path.split(path)[0]).mkdir(parents=True, exist_ok=True)
        cv2.imwrite('{}_{}.{}'.format(path[:-4], idx, 'jpg'), img) 
        
    def split_images(self, train_size=0.6, val_size=0.2):
        for class_name, image_list in self.image_paths.items():
            if class_name not in self.split_image_paths:
                self.split_image_paths[class_name] = {}
            self.split_image_paths[class_name]['train'] = image_list[:int(train_size*len(self.image_paths[class_name]))]
            self.split_image_paths[class_name]['val'] = image_list[int(train_size*len(self.image_paths[class_name])):int((train_size + val_size)*len(self.image_paths[class_name]))]
            self.split_image_paths[class_name]['test'] = image_list[int((train_size + val_size)*len(self.image_paths[class_name])):]

image_handler = ImageHandler(do_augmentation=True)

for file_path in zipfile.ZipFile('{}/SmartProducts.zip'.format(os.environ.get('ZIP_DIR')), 'r').namelist():
    tmp_image_list = []
    class_name, file_name = os.path.split(file_path)
    if class_name in CLASSES and not file_name.endswith('.txt'):
        tmp_image_list.append(file_name)
        image_handler.add_image(class_name=class_name, image=file_path)

image_handler.add_data_archive(path='{}/SmartProducts.zip'.format(os.environ.get('ZIP_DIR')))
image_handler.shuffle()
image_handler.split_images(train_size=0.6, val_size=0.2)
print('Splitting and Augmenting Files...')
image_handler.process('{}/split'.format(os.environ.get('DATA_DOWNLOAD_DIR')))
print('Finished')

## Show Images:

In [ ]:
import matplotlib.pyplot as plt
import random

def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def show_images(split, n=4):
    rows, columns = n, len(CLASSES)

    fig, axis = plt.subplots(rows, columns, figsize=(columns*5, rows*3))

    for c, cl in enumerate(CLASSES):
        images = os.listdir('./data/split/{}/{}/'.format(split, cl))
        random.shuffle(images)
        images = images[:rows]
        for j in range(rows):
            img = load_image(path='./data/split/{}/{}/{}'.format(split, cl, images[j]))
            axis[j, c].imshow(img)
            axis[j, c].axis('off')
            axis[j, c].set_title(cl, fontsize=20)

    plt.tight_layout()
    plt.show()

### Train:

In [ ]:
show_images(split='train', n=3)

### Val:

In [ ]:
show_images(split='val', n=3)

### Test:

In [ ]:
show_images(split='test', n=3)

### B. Download pretrained models <a class="anchor" id="head-1-2"></a>

In [ ]:
!ngc registry model list nvidia/tlt_pretrained_classification:*

In [ ]:
!mkdir -p $USER_EXPERIMENT_DIR/classification/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tlt_pretrained_classification:resnet18 --dest $USER_EXPERIMENT_DIR/classification/pretrained_resnet18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $USER_EXPERIMENT_DIR/classification/pretrained_resnet18/tlt_pretrained_classification_vresnet18

## 2. Provide training specfication <a class="anchor" id="head-2"></a>
* Training dataset
* Validation dataset
* Pre-trained models
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

In [ ]:
!cat $SPECS_DIR/classification_spec.cfg

## 3. Run TLT training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models

In [ ]:
!tlt-train classification -e $SPECS_DIR/classification_spec.cfg -r $USER_EXPERIMENT_DIR/classification/output -k $KEY

## 4. Evaluate trained models <a class="anchor" id="head-4"></a>

In this step, we assume that the training is complete and the model from the final epoch (`resnet_005.tlt`) is available. If you would like to run evaluation on an earlier model, please edit the spec file at `$SPECS_DIR/classification_spec.cfg` to point to the intended model.

In [ ]:
!tlt-evaluate classification -e $SPECS_DIR/classification_spec.cfg -k $KEY

## 5. Visualize Inferences <a class="anchor" id="head-5"></a>

To see the output results of our model on test images, we can use the `tlt-infer` tool. Note that using models trained for higher epochs will usually result in better results.

In [ ]:
%env EPOCH=005

Run inference in directory mode to run on a set of test images.  
**Choose from a Class which you specified at the beginning: For Example "kellogs_variety"**

In [ ]:
%env CLASS_TO_TEST=kamillentee

In [ ]:
!tlt-infer classification -m $USER_EXPERIMENT_DIR/classification/output/weights/resnet_$EPOCH.tlt \
                          -k $KEY -b 32 -d $DATA_DOWNLOAD_DIR/split/test/$CLASS_TO_TEST \
                          -cm $USER_EXPERIMENT_DIR/classification/output/classmap.json

As explained in Getting Started Guide, this outputs a results.csv file in the same directory. We can use a simple python program to see the visualize the output of csv file.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image 
import os
import csv
from math import ceil

DATA_DIR = os.environ.get('DATA_DOWNLOAD_DIR')
CLASS_TO_TEST = os.environ.get('CLASS_TO_TEST')
csv_path = os.path.join(DATA_DIR, 'split', 'test', CLASS_TO_TEST, 'result.csv')
results = []
with open(csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        results.append((row[0], row[1]))
        
columns = 5
rows = 3
fig = plt.figure(figsize=(columns*6, rows*6))
random.shuffle(results)
for i in range(1, columns*rows + 1):
    ax = fig.add_subplot(rows, columns,i)
    img = Image.open(results[i][0])
    plt.imshow(img)
    color = 'black'
    if CLASS_TO_TEST != results[i][1]:
        color = 'red'
    ax.set_title('Truth: {}\n Prediction: {}'.format(CLASS_TO_TEST, results[i][1]), fontsize=28, color=color)
    ax.axis('off')
plt.tight_layout()

## 6. Export and Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
!tlt-export classification \
            -m $USER_EXPERIMENT_DIR/classification/output/weights/resnet_$EPOCH.tlt \
            -o $USER_EXPERIMENT_DIR/classification/export/final_model.etlt \
            -k $KEY

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $USER_EXPERIMENT_DIR/classification/export/

### A. Int8 Optimization <a class="anchor" id="head-6-1"></a>
Classification model supports int8 optimization for inference in TRT. Inorder to use this, we must calibrate the model to run 8-bit inferences. This involves 2 steps

* Generate calibration tensorfile from the training data using tlt-int8-tensorfile
* Use tlt-export to generate int8 calibration table.

*Note: For this example, we generate a calibration tensorfile containing 10 batches of training data.
Ideally, it is best to use atleast 10-20% of the training data to calibrate the model.*

In [ ]:
!tlt-int8-tensorfile classification -e $SPECS_DIR/classification_spec.cfg \
                                    -m 10 \
                                    -o $USER_EXPERIMENT_DIR/classification/export/calibration.tensor

In [ ]:
# Remove the pre-existing exported .etlt file.
!rm -rf $USER_EXPERIMENT_DIR/classification/export/final_model.etlt
!tlt-export classification \
            -m $USER_EXPERIMENT_DIR/classification/output/weights/resnet_$EPOCH.tlt \
            -o $USER_EXPERIMENT_DIR/classification/export/final_model.etlt \
            -k $KEY \
            --cal_data_file $USER_EXPERIMENT_DIR/classification/export/calibration.tensor \
            --data_type int8 \
            --batches 10 \
            --cal_cache_file $USER_EXPERIMENT_DIR/classification/export/final_model_int8_cache.bin \
            -v 

### B. Generate TensorRT engine <a class="anchor" id="head-6-2"></a>
Verify engine generation using the `tlt-converter` utility included with the docker.

The `tlt-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tlt-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the converter for jetson from the dev zone link [here](https://developer.nvidia.com/tlt-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
!tlt-converter $USER_EXPERIMENT_DIR/classification/export/final_model.etlt \
               -k $KEY \
               -c $USER_EXPERIMENT_DIR/classification/export/final_model_int8_cache.bin \
               -o predictions/Softmax \
               -d 3,320,240 \
               -i nchw \
               -m 64 -t int8 \
               -e $USER_EXPERIMENT_DIR/classification/export/final_model.trt \
               -b 64